In [1]:
# import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys
import inspect
import time
import pickle

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

import src.simulation as s
import src.utils as utils

%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload

# 1. Peer-Nomination Network Data (Social Facilitation)


In [2]:
# instantiate simulation class

simulation =  s.Simulation()

/Users/davidblok/Work EUR/GitHub/ABM-PA/src/simulation.py:24: DtypeWarning: Columns (319,321,323,377,434,436,439,441,444,446,449,451,454,456,459,461,464,466,469,471,474,476,479,481,484,486,489,491) have mixed types.Specify dtype option on import or set low_memory=False.
  self.nominationPopulation = p.PeerNominatedDataPopulation('Peer-Nomination data population', self.input_args)
/Users/davidblok/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/davidblok/Work EUR/GitHub/ABM-PA/src/simulation.py:25: DtypeWarning: Columns (319,321,323,377,434,436,439,441,444,446,449,451,454,456,459,461,464,466,469,471,474,476,479,481,484,486,489,491) have mixed types.Specify dtype option on import or set low_memory=False.
  self.communicationPopulation = p.CommunicationDataPopulation('Communic

In [3]:
# Read tuned parameter combinations (See tuning_nom_soc_network.ipynb)
pars_nomination = pd.read_csv('../output/opt_pars_m_nomination_social.csv', sep=',', header=0)
pars_nomination

,threshold,ipa,error
0,0.0100,0.0025,0.000059
1,0.0075,0.0050,0.000066
2,0.0100,0.0050,0.000123
3,0.0075,0.0025,0.000224
4,0.0100,0.0075,0.000431
...,...,...,...
95,0.0475,0.0075,0.011245
96,0.0300,0.0175,0.011271
97,0.0400,0.0125,0.011279
98,0.0350,0.0150,0.011380


## 1. Pre-Intervention simulations

In [4]:
# Run pre-intervention for each parameter set
list_pre_results_child_nom = []
list_pre_results_class_nom = []
list_snapshot_pa_nom = []

for index, row in pars_nomination.iterrows():
    start = time.time()
    # run pre-intervention
    pre_results_nom, pre_results_avg_nom, snapshot_pa_nom = simulation.simulate_preinterventions(730, 'nomination_social', row['threshold'], row['ipa'])

    list_pre_results_child_nom.append(pre_results_nom)
    list_pre_results_class_nom.append(pre_results_avg_nom)
    list_snapshot_pa_nom.append(snapshot_pa_nom)

    end = time.time()
    print(index, row['threshold'], row['ipa'], "Time elapsed:", end - start, ' seconds')

# Save model output to object
file = open('../output/simulation/nomination_social/list_pre_results_child.obj', 'wb')
pickle.dump(list_pre_results_child_nom, file)

file = open('../output/simulation/nomination_social/list_pre_results_class.obj', 'wb')
pickle.dump(list_pre_results_class_nom, file)

file = open('../output/simulation/nomination_social/list_snapshot_pa.obj', 'wb')
pickle.dump(list_snapshot_pa_nom, file)


0 0.01 0.0025 Time elapsed: 7.947902202606201  seconds
1 0.0075 0.005 Time elapsed: 9.884315967559814  seconds
2 0.01 0.005 Time elapsed: 8.805680990219116  seconds
3 0.0075 0.0025 Time elapsed: 7.735826253890991  seconds
4 0.01 0.0075 Time elapsed: 7.556807994842529  seconds
5 0.0125 0.0025 Time elapsed: 7.500179052352905  seconds
6 0.005 0.037500000000000006 Time elapsed: 7.540108919143677  seconds
7 0.0125 0.005 Time elapsed: 7.350091934204102  seconds
8 0.0075 0.0075 Time elapsed: 7.15225887298584  seconds
9 0.015 0.0025 Time elapsed: 7.180596113204956  seconds
10 0.0125 0.0075 Time elapsed: 7.111523151397705  seconds
11 0.015 0.005 Time elapsed: 7.074594736099243  seconds
12 0.005 0.04 Time elapsed: 7.3773722648620605  seconds
13 0.0025 0.02 Time elapsed: 7.2792792320251465  seconds
14 0.017499999999999998 0.0025 Time elapsed: 7.153639316558838  seconds
15 0.005 0.0025 Time elapsed: 7.104973077774048  seconds
16 0.015 0.0075 Time elapsed: 7.154742956161499  seconds
17 0.0174999999

## 2. Intervention Simulations


In [5]:
effect = np.array([0.05, 0.1, 0.15, 0.17, 0.2])

for effect in effect:
    # Run interventions for each parameter combination
    list_results_child_nom = []
    list_results_class_nom = []
    list_influential_agents_nom = []

    for index, row in pars_nomination.iterrows():
        start = time.time()
        # run intervention
        results_nom, results_avg_nom, agents_per_intervention_nom = simulation.simulate_interventions(730, 'nomination_social', list_snapshot_pa_nom[index], row['threshold'], row['ipa'], effect, index)

        list_results_child_nom.append(results_nom)
        list_results_class_nom.append(results_avg_nom)
        list_influential_agents_nom.append(agents_per_intervention_nom)

        end = time.time()
        print(effect, index, row['threshold'], row['ipa'], "Time elapsed:", end - start, ' seconds')

    # Save model output to object
    name1 = ('../output/simulation/nomination_social/list_results_child_'+str(effect)+'.obj')
    file = open(name1, 'wb')
    pickle.dump(list_results_child_nom, file)

    name2 = ('../output/simulation/nomination_social/list_results_class_'+str(effect)+'.obj')
    file = open(name2, 'wb')
    pickle.dump(list_results_class_nom, file)

    name3 = ('../output/simulation/nomination_social/list_influential_agents_'+str(effect)+'.obj')
    file = open(name3, 'wb')
    pickle.dump(list_influential_agents_nom, file)

0.05 0 0.01 0.0025 Time elapsed: 43.009554862976074  seconds
0.05 1 0.0075 0.005 Time elapsed: 42.85618710517883  seconds
0.05 2 0.01 0.005 Time elapsed: 43.019367933273315  seconds
0.05 3 0.0075 0.0025 Time elapsed: 42.29625606536865  seconds
0.05 4 0.01 0.0075 Time elapsed: 42.269510984420776  seconds
0.05 5 0.0125 0.0025 Time elapsed: 42.34778308868408  seconds
0.05 6 0.005 0.037500000000000006 Time elapsed: 44.22452473640442  seconds
0.05 7 0.0125 0.005 Time elapsed: 44.82237982749939  seconds
0.05 8 0.0075 0.0075 Time elapsed: 47.75803804397583  seconds
0.05 9 0.015 0.0025 Time elapsed: 44.538230180740356  seconds
0.05 10 0.0125 0.0075 Time elapsed: 45.09889078140259  seconds
0.05 11 0.015 0.005 Time elapsed: 44.34440016746521  seconds
0.05 12 0.005 0.04 Time elapsed: 45.73348116874695  seconds
0.05 13 0.0025 0.02 Time elapsed: 45.63685393333435  seconds
0.05 14 0.017499999999999998 0.0025 Time elapsed: 44.175644874572754  seconds
0.05 15 0.005 0.0025 Time elapsed: 44.679327964782